In [1]:
import pandas as pd
import os
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import random

In [2]:
gt_train = pd.read_csv('train_GroundTruth2_binary.csv')
gt_train

,image_id,melanoma,seborrheic_keratosis
0,ISIC_0000002,1.0,0.0
1,ISIC_0000004,1.0,0.0
2,ISIC_0000013,1.0,0.0
3,ISIC_0000022,1.0,0.0
4,ISIC_0000026,1.0,0.0
...,...,...,...
623,ISIC_0015220,0.0,1.0
624,ISIC_0015233,0.0,1.0
625,ISIC_0015260,0.0,1.0
626,ISIC_0015284,1.0,0.0


In [4]:
image_train_folder = './train_binary/images'
masks_train_folder = './train_binary/masks'
aug_image_folder = './train_binary/aug_images'
aug_mask_folder = './train_binary/aug_masks'

In [5]:
# Create output folders if they don’t exist
os.makedirs(aug_image_folder, exist_ok=True)
os.makedirs(aug_mask_folder, exist_ok=True)

In [6]:
# Color jitter transform
color_jitter = transforms.ColorJitter(
    brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
)

In [7]:
# New rows for DataFrame
new_rows = []

In [8]:
# Loop over original images
for idx, row in gt_train.iterrows():
    image_id = row['image_id']
    image_path = os.path.join(image_train_folder, f"{image_id}.png")
    mask_path = os.path.join(masks_train_folder, f"{image_id}_segmentation.png")

    try:
        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        # Augmentations
        augmentations = {
            "hflip": lambda img, msk: (TF.hflip(img), TF.hflip(msk)),
            "vflip": lambda img, msk: (TF.vflip(img), TF.vflip(msk)),
            "rot90": lambda img, msk: (TF.rotate(img, 90), TF.rotate(msk, 90)),
            "cjitter": lambda img, msk: (color_jitter(img), msk),
        }

        for suffix, aug_func in augmentations.items():
            aug_image, aug_mask = aug_func(image, mask)

            # New ID
            new_id = f"{image_id}_{suffix}"

            # Save augmented image & mask
            aug_image.save(os.path.join(aug_image_folder, f"{new_id}.png"))
            aug_mask.save(os.path.join(aug_mask_folder, f"{new_id}_segmentation.png"))

            # Add to DataFrame
            new_rows.append({
                "image_id": new_id,
                "melanoma": row["melanoma"],
                "seborrheic_keratosis": row["seborrheic_keratosis"]
            })

    except Exception as e:
        print(f"Failed to process {image_id}: {e}")

In [9]:
# Append new rows to original DataFrame
augmented_df = pd.concat([gt_train, pd.DataFrame(new_rows)], ignore_index=True)
augmented_df

,image_id,melanoma,seborrheic_keratosis
0,ISIC_0000002,1.0,0.0
1,ISIC_0000004,1.0,0.0
2,ISIC_0000013,1.0,0.0
3,ISIC_0000022,1.0,0.0
4,ISIC_0000026,1.0,0.0
...,...,...,...
3135,ISIC_0015284_cjitter,1.0,0.0
3136,ISIC_0015295_hflip,0.0,1.0
3137,ISIC_0015295_vflip,0.0,1.0
3138,ISIC_0015295_rot90,0.0,1.0


In [10]:
# Save updated DataFrame
augmented_df.to_csv('train_GroundTruth2_binary_augmented.csv', index=False)